<a href="https://colab.research.google.com/github/shinchan75034/tensorflow-pocket-ref/blob/main/chapter10/OReilly_C10_CIFAR10_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
import tensorflow as tf
import kerastuner as kt
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pylab as plt
import os
from datetime import datetime

In [ ]:
from datetime import datetime

In [ ]:
print(tf.__version__)

2.4.1


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

170500096/170498071 [==============================] - 3s 0us/step


In [ ]:
# Plain text name in alphabetical order. https://www.cs.toronto.edu/~kriz/cifar.html
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 
               'deer','dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
validation_dataset = tf.data.Dataset.from_tensor_slices((test_images[:500], test_labels[:500]))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images[500:], test_labels[500:]))

In [ ]:
# Create an instance of dataset from raw numpy images and labels.
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#transformations_2
train_dataset_size = len(list(train_dataset.as_numpy_iterator()))
print('Training data sample size: ', train_dataset_size)

validation_dataset_size = len(list(validation_dataset.as_numpy_iterator()))
print('Validation data sample size: ', validation_dataset_size)

test_dataset_size = len(list(test_dataset.as_numpy_iterator()))
print('Test data sample size: ', test_dataset_size)

Training data sample size:  50000
Validation data sample size:  500
Test data sample size:  9500


## Define a distribution strategy
Create a `MirroredStrategy` object to handle distributed training.

In [ ]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [ ]:
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
train_dataset = train_dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE).batch(validation_dataset_size)
test_dataset = test_dataset.batch(test_dataset_size)


In [ ]:
STEPS_PER_EPOCH = train_dataset_size // BATCH_SIZE_PER_REPLICA
VALIDATION_STEPS = 1

In [ ]:

def build_model(hp):
  model = tf.keras.Sequential()
  # Node count for next layer as hyperparameter
  hp_node_count = hp.Int('units', min_value=16, max_value=32, step=8)
  model.add(tf.keras.layers.Conv2D(filters = hp_node_count, 
                                   kernel_size=(3, 3), 
                                   activation='relu', 
                                   name = 'conv_1',
                                   kernel_initializer='glorot_uniform', 
                                   padding='same', input_shape = (32,32,3)))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Flatten(name = 'flat_1'))
  # Activation function for next layer as hyperparameter
  hp_AF = hp.Choice('dense_activation', values = ['relu', 'tanh'])
  model.add(tf.keras.layers.Dense(256, activation=hp_AF, 
                                  kernel_initializer='glorot_uniform', 
                                  name = 'dense_1'))
  model.add(tf.keras.layers.Dense(10, activation='softmax', 
                                  name = 'custom_class'))

  
  model.build([None, 32, 32, 3])
  # Compile model with optimizer 
  # Learning rate as hyperparameter
  hp_LR = hp.Float('learning_rate', 1e-2, 1e-4)
  
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_LR),
      metrics=['accuracy'])
  
  return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
!ls -lrt ./my_dir/intro_to_kt 

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
import time
import datetime

ts = time.time()
print(ts) # finish epoch time in second 
datetime_time = datetime.datetime.fromtimestamp(ts)
print(datetime_time)

In [ ]:
tuner.search(train_dataset, 
             steps_per_epoch = STEPS_PER_EPOCH,
             validation_data = validation_dataset,
             validation_steps = VALIDATION_STEPS,
             epochs = 15,
             callbacks = [early_stop]
             )

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in conv_1
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')} and the optimal activation for dense_1 layer
is {best_hps.get('dense_activation')}.
""")

In [ ]:
ts = time.time()
print(ts) # finish epoch time in second 
datetime_time = datetime.datetime.fromtimestamp(ts)
print(datetime_time)

In [ ]:
#Started in 5:28 pm CDT. 1617402511

In [ ]:
1617403117 - 1617402511
# 10 minutes running time.

In [ ]:
best_hps.get('units')

In [ ]:
best_hps.get('learning_rate')

In [ ]:
best_hps.get('dense_activation')

## Launch full training with best hyperparameters

In [ ]:
best_hp_model = tuner.hypermodel.build(best_hps)

In [ ]:
MODEL_NAME = 'myCIFAR10-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print(MODEL_NAME)
checkpoint_dir = './' + MODEL_NAME
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt-{epoch}")
print(checkpoint_prefix)

In [ ]:
myCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='val_accuracy',
    mode='max',
    save_weights_only = True,
    save_best_only = True
    )

In [ ]:
best_hp_model.fit(train_dataset, 
             steps_per_epoch = STEPS_PER_EPOCH,
             validation_data = validation_dataset,
             validation_steps = VALIDATION_STEPS,
             epochs = 15,
             callbacks = [early_stop, myCheckPoint])

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
best_hp_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))